In [1]:
def accumulate_return_to_month(step, accumulate_return):
    return (accumulate_return+1.)**(1./step) - 1. 

In [120]:
def summary_investment(price_list, position_list, trading_fee, outer_trading_fee=True):
    price_list = [float(i) for i in price_list]
    position_list = [float(i) for i in position_list]
    accumulate_return = 0
    enter_times = 0
    hold_times = 0
    buy_price = 0
    average_buy_price = 0
    sell_price = 0    
    volatility_list = []
    total_cost = 0
    over_buy_price_times = 0
    total_profit = 0
    Return = 0
    cost = 0
    for i in range(len(price_list)):
        if i==0 and position_list[i]!=0:
            buy_price = price_list[i]
            average_buy_price = buy_price
            if outer_trading_fee :
                cost = buy_price*position_list[i]*(1+trading_fee)
            else :
                cost = buy_price*position_list[i]/(1-trading_fee)
            total_cost += cost
            total_profit -= cost
            enter_times += 1
        elif position_list[i] > position_list[i-1]:
            #buy
            buy_price = price_list[i]
            diff_position = position_list[i]-position_list[i-1]
            if outer_trading_fee :
                cost = buy_price*diff_position*(1+trading_fee)
            else :
                cost = buy_price*diff_position/(1-trading_fee)       
            total_cost += cost
            total_profit -= cost
            average_buy_price = total_cost/position_list[i]
            enter_times += 1
        elif position_list[i] < position_list[i-1]:
            #sell
            sell_price = price_list[i]
            diff_position = position_list[i-1]-position_list[i]
            total_profit += sell_price*diff_position
#             Return = (sell_price-average_buy_price)/(average_buy_price) - trading_fee
#             Return = total_profit / float(total_cost)
#             accumulate_return = (accumulate_return+1)*(Return+1)-1
#             total_cost = 0

        if i!=len(price_list)-1 and position_list[i]!=0:
            if price_list[i+1]>average_buy_price :
                over_buy_price_times += 1
            hold_times += 1
            volatility_list.append(price_list[i+1] / price_list[i]-1)
        elif i!=len(price_list)-1 and position_list[i]==0 :
            volatility_list.append(0)
#         print total_profit
#         print price_list[i],buy_price,average_buy_price,sell_price,Return,accumulate_return,enter_times,hold_times,over_buy_price_times
#     print volatility_list,np.std(volatility_list,ddof=1)
    accumulate_return = total_profit/float(total_cost)
    if hold_times !=0:
        over_buy_price_prob = over_buy_price_times/float(hold_times)
    else:
        over_buy_price_prob = np.nan
    volatility = np.std(volatility_list,ddof=1)
    volatility_down_side = np.std([i if i<0 else 0 for i in volatility_list],ddof=1)
    if volatility != 0:
        sharpe = accumulate_return/volatility
    else :
        sharpe = 0

    reslut_dict = {'accumulate_return':accumulate_return,
                   'total_profit':total_profit,
                   'enter_times':enter_times,
                   'over_buy_price_prob':over_buy_price_prob,
                   'hold_times':hold_times,
                   'volatility':volatility,
                   'volatility_down_side':volatility_down_side,
                   'sharpe':sharpe}
    return reslut_dict

In [85]:
def trend (price, predict_months_price, threshold, noise ) :
    if np.isnan(predict_months_price).any() : 
        return np.nan
    else:
        trend_ans = [(i-price)/float(price) > threshold for i in predict_months_price] 
        if noise :
            #1:有改變,0:沒改變
            noise_list = [np.random.binomial(1, i) for i in noise]
            trend = [ (trend_ans[i] + noise_list[i])%2 for i in range(len(trend_ans))]
        else:
            trend = trend_ans
        trend = [-1 if i==0 else 1 for i in trend]
        return trend

In [108]:
def trend_normal_random (price, predict_months_price, threshold, noise ) :
    if np.isnan(predict_months_price).any() : 
        return np.nan
    else:
        trend_ans = [(i-price)/float(price) > threshold for i in predict_months_price] 
        if noise :
            #1:有改變,0:沒改變
            simulate_noise = [np.random.normal(loc=i[0],scale=i[1]) for i in noise]
#             print simulate_noise
            simulate_noise = [0 if i<0 else 1 if i>1 else i for i in simulate_noise]
#             print simulate_noise
#             print trend_ans
            noise_list = [np.random.binomial(1, i) for i in simulate_noise]
            trend = [ (trend_ans[i] + noise_list[i])%2 for i in range(len(trend_ans))]
        else:
            trend = trend_ans
        trend = [-1 if i==0 else 1 for i in trend]
        return trend    

In [2]:
# def data_row_gap(data,gap,miss_fill='ffill'):
#     return data.replace(0, np.nan).fillna(method='ffill')[::gap]

In [3]:
def split_train_test_data(data, start_pre_date=None, history_length=None, trend_months=None):
    if trend_months is None:
        train_data=data.loc[data.tx_month<start_pre_date].reset_index(drop=True)
        test_data=data[data.loc[data.tx_month>=start_pre_date].index[0]-(history_length-1):].reset_index(drop=True)
    else:
        train_data=data.loc[data.tx_month<future_date(start_pre_date, [max(trend_months)])[0]].reset_index(drop=True)
        test_data=data[data.loc[data.tx_month>=start_pre_date].index[0]-(history_length-1):].reset_index(drop=True)
    return train_data,test_data

In [4]:
def future_date(date, future_month):
    
    year, month, day = [int(i) for i in date.split('-')]
#     print year, month, day
    future_date_list=[]
    for i in future_month:
        if float((month+i-12))/12>0:
            new_year = year + (month+i-12)/12
        else : 
            new_year = year
        if (month+i)%12==0:
            new_month = 12
        else :
            new_month = (month+i)%12
#         print new_year, new_month, day, float((month+i))/12, (month+i)/12, i
        future_date_list = future_date_list + [date_to_str(new_year, new_month, day)]
    return future_date_list

In [5]:
def date_to_str(year, month, day):
    if month<10:
        month = '0' + str(month)
    else :
        month = str(month)
    if day<10:
        day = '0' + str(day)
    else :
        day = str(day)
    return str(year) + '-' + month + '-' + day

In [6]:
# def split_data(x,index,forward=True):
#     if forward :
#         return x[:,:,:index]
#     else :
#         return x[:,:,index:]

In [1]:
def split_data(x,index):
    return x[:,:,index[0]:index[1]]

In [8]:
# def history_data(data, env, history_length):
#     d={'history_accumulation_return':env._history_accumulation_return,
#        'history_action':env._history_action,
#        'history_return':env._history_return,
#        'history_diff':env._history_diff,
#        'history_positionn':env._history_position}
#     D=pd.DataFrame(d)
#     history_data=pd.concat([data[history_length:].reset_index(drop=True),D],axis=1)
#     return history_data

In [9]:
def summary_result(data,action_list,trading_fee,last_day_sell=False):
    data = data.reset_index(drop=True)
    if last_day_sell==True:
        data['action'] = action_list + [2]
    else:
        data['action'] = action_list
    position=0
    position_list = []
    Return_list = []
    Cumulative_Return = 0
    Cumulative_Return_list = []
    Diff_list = []
    Cumulative_Diff = 0
    Cumulative_Diff_list = []  
    buy_price_list = []
    for i in range(len(data)):
        Return = 0
        Diff = 0
        if data.action[i]==1:
            
            buy_price_list.append(data.close[i]) 
            position += 1

        elif (position>=1) and data.action[i]==2:
            buy_price_mean = np.mean(buy_price_list)
            sell_price = data.close[i]
            Return = ( sell_price - buy_price_mean ) / buy_price_mean - trading_fee
            Cumulative_Return = ( 1 + Cumulative_Return ) * ( 1 + Return ) - 1 
            Diff = ( sell_price - buy_price_mean - buy_price_mean * trading_fee ) * position 
            Cumulative_Diff += Diff
            position = 0
            buy_price_list = []
        position_list.append(position)
        Return_list.append(Return)
        Cumulative_Return_list.append(Cumulative_Return)
        Diff_list.append(Diff)
        Cumulative_Diff_list.append(Cumulative_Diff)
    data['position'] = position_list
    data['Return'] = Return_list
    data['Cumulative_Return'] = Cumulative_Return_list
    data['Diff'] = Diff_list
    data['Cumulative_Diff'] = Cumulative_Diff_list
    benchmark_buy_price = data.dropna().head(1).close.values[0]
    benchmark_sell_price = data.dropna().tail(1).close.values[0]
    benchmark_return = ( benchmark_sell_price - benchmark_buy_price ) / benchmark_buy_price - trading_fee
    benchmark_diff = benchmark_sell_price - benchmark_buy_price - benchmark_buy_price * trading_fee
    print 'benchmark buy price : ', benchmark_buy_price
    print 'benchmark_sell_price : ', benchmark_sell_price
    print 'benchmark_return : ', benchmark_return
    print 'benchmark_diff : ', benchmark_diff
    return data